In [58]:
import sys, os
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import glob
import re
import requests
import pandas as pd

In [2]:
def xcmd(cmd, verbose=False):
    
    if verbose: print('\n'+cmd)

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if failure:
        print('execution of %s failed' % cmd)
        print('error is as follows',output)
        sys.exit()
    else:
        return output

In [3]:
def mySoup(my_url):
    
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")    
    
    return page_soup


In [100]:
# based on 
# https://ned.ipac.caltech.edu/forms/gmd.html

In [110]:
def fix_name(name):
    return "+".join([x.upper() for x in name.strip().rsplit(" ") if x!=""])

def get_ned_info(name_list):

    query = ""
    for i, name in enumerate(name_list):
        if i==0:
            query += fix_name(name)
        else:
            query = query + "%0D%0A" + fix_name(name)



    # query = "UGC+12517%0D%0ANGC+2557"
    my_url = "https://ned.ipac.caltech.edu/cgi-bin/gmd?uplist={}&delimiter=bar&NO_LINKS=1&nondb=row_count&nondb=user_name_msg&nondb=user_objname&crosid=objname&position=ra%2Cdec&position=gallon%2Cgallat&position=bhextin&attdat=attned&gphotoms=q_value&gphotoms=q_unc&gphotoms=ned_value&gphotoms=ned_unc&diamdat=ned_maj_dia&distance=avg&distance=stddev_samp".format(query)
    page_soup = mySoup(my_url)

    rows = page_soup.findAll("pre")[0].text.split("\n")

    columns = None
    output = {}


    for i, row in enumerate(rows):

        cols = row.split("|")
        cl = []
        for c in cols:
            cc = c.strip()
            if cc != "":
                if cc == "Burstein":
                    cc = "Extinction"
                cl.append(cc)

        try:
            if cl[0] == "Row":
                columns = cl
        except:
            pass
        try:
            if int(cl[0])>0:
                output[cl[1]] = cl

        except:
            pass
        
    if columns and output:
        return pd.DataFrame.from_dict(output, orient="index", columns=columns)

    return None


In [111]:
name_list = [
    "UGC 12517",
    "NGC 2557",
    "pgc 2557",
    ]

get_ned_info(name_list)

,Row,Input Object Name,Object Name,RA,Dec,Gal,Gal,Extinction,Row
UGC 12517,1,UGC 12517,UGC 12517,23h19m54.44s,+43d57m27.8s,106.013,-15.886,0.753,1
NGC 2557,2,NGC 2557,NGC 2557,08h19m10.75s,+21d26m08.8s,202.005,28.432,0.121,2
PGC 2557,3,PGC 2557,MESSIER 031,00h42m44.35s,+41d16m08.6s,121.174,-21.573,0.301,3


In [116]:
name_list = [
    "m31",
    "n2418"
    ]

df = get_ned_info(name_list)
df

,Row,Input Object Name,Object Name,RA,Dec,Gal,Gal,Extinction,Row
M31,1,M31,MESSIER 031,00h42m44.35s,+41d16m08.6s,121.174,-21.573,0.301,1
N2418,2,N2418,NGC 2418,07h36m37.51s,+17d53m02.1s,201.525,17.815,0.109,2


In [52]:
page_soup.findAll("h2")[0].findAll("font")[0].text

'1 of 1 detected input names were recognized by the NED name interpreter.'

In [124]:
ra = df.iloc[0]["RA"]
dec = df.iloc[0]["Dec"]
ra, dec

('00h42m44.35s', '+41d16m08.6s')

In [119]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [128]:
def to_degree(RA, Dec):
    c = SkyCoord(RA, Dec)
    return c.ra.degree, c.dec.degree
    

In [135]:
df["ra_dec"] = df.apply(lambda row: to_degree(row.RA, row.Dec), axis=1)
df

,Row,Input Object Name,Object Name,RA,Dec,Gal,Gal,Extinction,Row,x,ra_dec
M31,1,M31,MESSIER 031,00h42m44.35s,+41d16m08.6s,121.174,-21.573,0.301,1,"(10.684791666666664, 41.26905555555555)","(10.684791666666664, 41.26905555555555)"
N2418,2,N2418,NGC 2418,07h36m37.51s,+17d53m02.1s,201.525,17.815,0.109,2,"(114.15629166666665, 17.883916666666668)","(114.15629166666665, 17.883916666666668)"


In [136]:
ra, dec = df.iloc[0]["ra_dec"]

In [137]:
ra, dec

(10.684791666666664, 41.26905555555555)